<a href="https://colab.research.google.com/github/theschool8/transportation_problem/blob/main/Transportation_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks//Calculation OPTIMIZATION Nett Margin.xlsx'
data_cost = pd.read_excel(path, sheet_name=3, index_col=0)
data_cost.head()

,nett,cost,plant,branch,type,capacity
0,46500,3500,0,BANDUNG,AA,15000
1,47500,2500,0,YOGYA,AA,15000
2,47000,3000,0,MALANG,AA,15000
3,47000,3000,0,JAKARTA,AA,15000
4,41500,3500,0,BANDUNG,BB,20000


In [ ]:
data_demand = pd.read_excel(path, sheet_name=0, usecols=range(1,4), 
                            nrows=12, names=['demand', 'branch', 'type'])
data_demand

,demand,branch,type
0,20000,BANDUNG,AA
1,12000,BANDUNG,BB
2,12000,BANDUNG,CC
3,15000,YOGYA,AA
4,11000,YOGYA,BB
5,10500,YOGYA,CC
6,15000,MALANG,AA
7,8500,MALANG,BB
8,9000,MALANG,CC
9,12000,JAKARTA,AA


In [ ]:
data_all = pd.merge(data_cost, data_demand, on=['branch', 'type'], how='left')
data_all['margin'] = data_all[['cost','nett']].sum(axis=1)
data_all['branch_id'] = data_all['branch']
code = {'branch_id': {'BANDUNG': 0, 'YOGYA': 1, 'MALANG': 2, 'JAKARTA': 3}}
data_all = data_all.replace(code)
data_all.head(10)

,nett,cost,plant,branch,type,capacity,demand,margin,branch_id
0,46500,3500,0,BANDUNG,AA,15000,20000,50000,0
1,47500,2500,0,YOGYA,AA,15000,15000,50000,1
2,47000,3000,0,MALANG,AA,15000,15000,50000,2
3,47000,3000,0,JAKARTA,AA,15000,12000,50000,3
4,41500,3500,0,BANDUNG,BB,20000,12000,45000,0
5,42500,2500,0,YOGYA,BB,20000,11000,45000,1
6,42000,3000,0,MALANG,BB,20000,8500,45000,2
7,42000,3000,0,JAKARTA,BB,20000,7000,45000,3
8,56500,3500,0,BANDUNG,CC,12000,12000,60000,0
9,57500,2500,0,YOGYA,CC,12000,10500,60000,1


In [ ]:
data = {}
type = ['AA', 'BB', 'CC']
for i in type:
  data[i] = data_all[data_all['type'] == i]

## Balancing the data 
* if supply > demand then add the dummy column
* if supply < demand then add the dummy row




In [ ]:
def balancing(data, supply, demand, supply_id, demand_id, cost, penalties = None):
  total_supply = sum(data.groupby(supply_id)[supply].min())
  total_demand = sum(data.groupby(demand_id)[demand].min())
  nrow = data.groupby(supply_id).count().shape[0]
  ncol = data.groupby(demand_id).count().shape[0]
  # supply and demand is already balance
  if total_supply == total_demand:
    return data[[supply_id, demand_id, supply, demand, cost]]
  # demand is greater than supply, 
  # default penalties cost is 0
  if total_supply < total_demand:
    for i in range(ncol):
      new_supply_id = nrow 
      new_demand_id = i
      new_supply = total_demand - total_supply
      new_demand = data[data[demand_id] == i][demand].min()
      if penalties is None:
        new_cost = 0
      else:
        new_cost = penalties[i]
      data = data.append({supply_id: new_supply_id,
                          demand_id: new_demand_id,
                          supply: new_supply,
                          demand: new_demand,
                          cost: new_cost}, ignore_index=True)
    return data[[supply_id, demand_id, supply, demand, cost]]
    # supply is greater than demand
    # no penalties cost because they dont lose the consumer
  if total_supply > total_demand:
    for i in range(nrow):
      new_supply_id = i 
      new_demand_id = ncol
      new_supply = data[data[supply_id] == i][supply].min()
      new_demand = total_supply - total_demand
      new_cost = 0
      data = data.append({supply_id: new_supply_id,
                          demand_id: new_demand_id,
                          supply: new_supply,
                          demand: new_demand,
                          cost: new_cost}, ignore_index=True)
    return data[[supply_id, demand_id, supply, demand, cost]]

#Using Least Cost Method

In [ ]:
# Least Cost Method
def LeastCost(data, supply, demand, supply_id, demand_id, cost, obj='min'):
  data1 = data.copy()
  # if the objective is to maximize problem (obj='max')
  ## then we convert cost value by substract all data by highest cost value
  ### then it become minimization problem as usual
  if obj == 'max':
    data1[cost] = data1[cost] - max(data1[cost])
    data1[cost] = data1[cost].abs()
  if obj != 'max' and obj != 'min':
    raise Exception("use objective 'min' [default] for minimalization or 'max' for maximization problem")  
  
  data2 = balancing(data1, supply, demand, supply_id, demand_id, cost)
  data2['optimize'] = 0 
  n = data2.shape[0] 
  for i in range(n):
    # finding minimum cost value
    minval = min(data2[cost])
    minval_df = data2[data2[cost] == minval]
    # find optimal allocation
    min_alo = minval_df[[supply,demand]].min(axis=1)
    alo = min_alo.nsmallest(1)
    # assign allocation
    data2.loc[alo.index, 'optimize'] = alo.values[0]
    # updating supply
    row = data2.loc[alo.index, supply_id].values[0]
    row_idx = data2[data2[supply_id] == row].index
    new_supply = data2.loc[alo.index, supply].values[0] - alo.values[0]
    data2.loc[row_idx, supply] = new_supply
    # updating demand
    col = data2.loc[alo.index, demand_id].values[0]
    col_idx = data2[data2[demand_id] == col].index
    new_demand = data2.loc[alo.index, demand].values[0] - alo.values[0]
    data2.loc[col_idx, demand] = new_demand
    # remove cost 
    ## using infinity number to prevent chosen by minimum value
    data2.loc[alo.index, cost] = np.inf 
  # return origin data before transformation for maximize problem obj='max'
  data3 = balancing(data, supply, demand, supply_id, demand_id, cost)
  return pd.concat([data3, data2['optimize']], axis=1)

In [ ]:
# Run the Least Cost method to All Data
output = {'cost':{}, 'margin':{}, 'nett':{}}

for keys in output:
  LC = {}
  nett_margin = {}
  table_opt = {}
  type = ['AA','BB','CC']
  for i in type:
    if keys == 'cost':
      objj = 'min'
    else:
      objj = 'max'
    LC[i] = LeastCost(data[i], supply='capacity', demand='demand', 
                    supply_id='plant', demand_id='branch_id', cost=keys, obj=objj)
    table_opt[i] = pd.pivot_table(data=LC[i], index='plant', 
                                  columns='branch_id', values='optimize')
    margin_df = LC[i].merge(data[i], how='inner')
    # Calculate total net margin
    margin_df['calc'] = margin_df['optimize'] * margin_df['nett']
    nett_margin[i] = margin_df['calc'].sum()
  output[keys] = {'LC': LC, 
                  'table_opt': table_opt, 
                  'nett_margin': nett_margin}

## Output Base of Minimum Cost

In [ ]:
output['cost']['nett_margin']

{'AA': 2902800000.0, 'BB': 1708700000.0, 'CC': 2381150000.0}

In [ ]:
# Optimal allocation table for AA product
output['cost']['table_opt']['AA']

branch_id,0.0,1.0,2.0,3.0
plant,,,,
0.0,0.0,0.0,3000.0,12000.0
1.0,0.0,15000.0,7000.0,0.0
2.0,15000.0,0.0,5000.0,0.0
3.0,5000.0,0.0,0.0,0.0


In [ ]:
output['cost']['table_opt']['BB']

branch_id,0.0,1.0,2.0,3.0,4.0
plant,,,,,
0.0,0.0,0.0,6500.0,7000.0,6500.0
1.0,2000.0,11000.0,2000.0,0.0,0.0
2.0,10000.0,0.0,0.0,0.0,0.0


In [ ]:
output['cost']['table_opt']['CC']

branch_id,0.0,1.0,2.0,3.0,4.0
plant,,,,,
0.0,0.0,0.0,0.0,0.0,12000.0
1.0,0.0,10500.0,4000.0,0.0,500.0
2.0,12000.0,0.0,5000.0,10000.0,0.0


In [ ]:
# total nett margin
sum(output['cost']['nett_margin'].values())

6992650000.0

## Output Base of Maximum Margin

In [ ]:
output['margin']['nett_margin']

{'AA': 2876300000.0, 'BB': 1670300000.0, 'CC': 2359300000.0}

In [ ]:
output['margin']['table_opt']['AA']

branch_id,0.0,1.0,2.0,3.0
plant,,,,
0.0,7000.0,8000.0,0.0,0.0
1.0,0.0,7000.0,15000.0,0.0
2.0,8000.0,0.0,0.0,12000.0
3.0,5000.0,0.0,0.0,0.0


In [ ]:
output['margin']['table_opt']['BB']

branch_id,0.0,1.0,2.0,3.0,4.0
plant,,,,,
0.0,4000.0,1000.0,8500.0,0.0,6500.0
1.0,8000.0,0.0,0.0,7000.0,0.0
2.0,0.0,10000.0,0.0,0.0,0.0


In [ ]:
output['margin']['table_opt']['CC']

branch_id,0.0,1.0,2.0,3.0,4.0
plant,,,,,
0.0,0.0,0.0,0.0,0.0,12000.0
1.0,6000.0,0.0,9000.0,0.0,0.0
2.0,6000.0,10500.0,0.0,10000.0,500.0


In [ ]:
sum(output['margin']['nett_margin'].values())

6905900000.0

## Output Base of Maximum Nett Margin

In [ ]:
output['nett']['nett_margin']

{'AA': 2899300000.0, 'BB': 1708500000.0, 'CC': 2341550000.0}

In [ ]:
output['nett']['table_opt']['AA']

branch_id,0.0,1.0,2.0,3.0
plant,,,,
0.0,0.0,0.0,10000.0,5000.0
1.0,0.0,15000.0,0.0,7000.0
2.0,15000.0,0.0,5000.0,0.0
3.0,5000.0,0.0,0.0,0.0


In [ ]:
output['nett']['table_opt']['BB']

branch_id,0.0,1.0,2.0,3.0,4.0
plant,,,,,
0.0,2000.0,0.0,4500.0,7000.0,6500.0
1.0,0.0,11000.0,4000.0,0.0,0.0
2.0,10000.0,0.0,0.0,0.0,0.0


In [ ]:
output['nett']['table_opt']['CC']

branch_id,0.0,1.0,2.0,3.0,4.0
plant,,,,,
0.0,0.0,0.0,4000.0,0.0,8000.0
1.0,0.0,10500.0,0.0,0.0,4500.0
2.0,12000.0,0.0,5000.0,10000.0,0.0


In [ ]:
sum(output['nett']['nett_margin'].values())

6949350000.0

# Using Vogel Aproximation Method (VAM)

In [ ]:
def find_diff(data, cost, column_id, column, n):
  data['p'] = -np.inf
  for i in range(n):
    initial = data[data[column_id] == i]
    if min(initial[column]) != 0:
      two_small = initial.nsmallest(2, cost)
      two_list = two_small[cost].tolist()
      two_small_diff = two_list[1]- two_list[0]
      data.loc[initial.index, 'p'] = two_small_diff
    else:
      data.loc[initial.index, 'p'] = -np.inf
  return data[['p']]

# function to create penalties cost by row (supply) & column (demand)
def penalties(data, supply_id, demand_id, supply, demand,cost): 
  copy = data.copy()
  nrow = copy.groupby(supply_id).count().shape[0]
  ncol = copy.groupby(demand_id).count().shape[0]
  a = find_diff(copy, cost, supply_id, supply, nrow)
  b = find_diff(copy, cost, demand_id, demand, ncol)
  ab = pd.concat([a,b], axis=1)
  ab.columns = ['p_row', 'p_col']
  return ab

In [ ]:
# Vogel Aproximation Method
def VAM(data, supply, demand, supply_id, demand_id, cost, obj='min'):
  data1 = data.copy()
  # if the objective is to maximize problem (obj='max')
  ## then we convert cost value by substract all data by highest cost value
  ### then it become minimization problem as usual
  if obj == 'max':
    data1[cost] = data1[cost] - max(data1[cost])
    data1[cost] = data1[cost].abs()
  if obj != 'max' and obj != 'min':
    raise Exception("use objective 'min' [default] for minimalization or 'max' for maximization problem")
  
  data2 = balancing(data1, supply, demand, supply_id, demand_id, cost)
  n = data2.shape[0]
  data2['optimize'] = 0
  for i in range(n):
    penalty = penalties(data2, supply_id, demand_id, supply, demand, cost)
    vogel = pd.concat([data2,penalty], axis=1)
    # find maximum penalties of row table & column table
    max_prow = max(vogel['p_row'])
    max_pcol = max(vogel['p_col'])
    # there 2 condition, which is:
    # 1.if maximum penalties of row table (supply) is not equal than maximum penalties of row table (supply)
    # 2.otherwise
    if max_prow != max_pcol:
      if max_prow > max_pcol: 
        # find max penalties of row
        find_pmax = vogel[vogel['p_row'] == max_prow]
        # find min cost 
        find_cmin = find_pmax[find_pmax[cost] == min(find_pmax[cost])]
        # find optimal alocation
        min_alo = find_cmin[[supply,demand]].min(axis=1)
        alo = min_alo.nsmallest(1)
      else:
        # find max penalties of column
        find_pmax = vogel[vogel['p_col'] == max_pcol]
        # find min cost 
        find_cmin = find_pmax[find_pmax[cost] == min(find_pmax[cost])]
        # find optimal alocation
        min_alo = find_cmin[[supply,demand]].min(axis=1)
        alo = min_alo.nsmallest(1)

    # if penalty row & penalty cost are equal
    if max_prow == max_pcol: 
      # find max penalties & min cost of row 
      find_pmax_r = vogel[vogel['p_row'] == max_prow]
      cmin_r = min(find_pmax_r[cost])
      # find max penalties & min cost of column 
      find_pmax_c = vogel[vogel['p_col'] == max_pcol]
      cmin_c = min(find_pmax_c[cost])
      # there are 3 condition of possible cost
      if cmin_r == cmin_c: # 1.if cost of penalty row & penalty column are equal
        find_cmin_r = find_pmax_r[find_pmax_r[cost] == cmin_r]
        min_alo_r = find_cmin_r[[supply,demand]].min(axis=1)
        alo_r = min_alo_r.nsmallest(1)
        find_cmin_c = find_pmax_c[find_pmax_c[cost] == cmin_c]
        min_alo_c = find_cmin_c[[supply,demand]].min(axis=1)
        alo_c = min_alo_c.nsmallest(1)
        if alo_r.values[0] > alo_c.values[0]:
          alo = alo_r
        else :
          alo = alo_c
      elif cmin_r < cmin_c: # 2.if cost of penalty row < penalty column 
        find_cmin = find_pmax_r[find_pmax_r[cost] == cmin_r]
        min_alo = find_cmin[[supply,demand]].min(axis=1)
        alo = min_alo.nsmallest(1)
      else: # 3.if cost of penalty row > penalty column
        find_cmin = find_pmax_c[find_pmax_c[cost] == cmin_c]
        min_alo = find_cmin[[supply,demand]].min(axis=1)
        alo = min_alo.nsmallest(1)
    # assign allocation
    vogel.loc[alo.index, 'optimize'] = alo.values[0]  
    # updating supply
    row = vogel.loc[alo.index, supply_id].values[0]
    row_idx = vogel[vogel[supply_id] == row].index
    new_supply = vogel.loc[alo.index, supply].values[0] - alo.values[0]
    vogel.loc[row_idx, supply] = new_supply
    # updating demand
    col = vogel.loc[alo.index, demand_id].values[0]
    col_idx = vogel[vogel[demand_id] == col].index
    new_demand = vogel.loc[alo.index, demand].values[0] - alo.values[0]
    vogel.loc[col_idx, demand] = new_demand
    # remove cost 
    vogel.loc[alo.index, cost] = np.inf
    data2 = vogel[[supply_id, demand_id, supply, demand, cost,'optimize']]
  data3 = balancing(data.copy(), supply, demand, supply_id, demand_id, cost)
  return pd.concat([data3, data2['optimize']], axis=1)

In [ ]:
# run VAM method to All Data
output2 = {'cost':{}, 'margin':{}, 'nett':{}}

for keys in output2:
  vogel = {}
  nett_margin = {}
  table_opt = {}
  type = ['AA', 'BB', 'CC']
  for i in type:
    if keys == 'cost':
      objj = 'min'
    else:
      objj = 'max'
    vogel[i] = VAM(data[i], supply='capacity', demand='demand', 
                   supply_id='plant', demand_id='branch_id', 
                   cost=keys, obj=objj)
    table_opt[i] = pd.pivot_table(data=vogel[i], index='plant', 
                                  columns='branch_id', values='optimize')
    margin_df = vogel[i].merge(data[i], how='inner')
    # Calculate total net margin
    margin_df['calc'] = margin_df['optimize'] * margin_df['nett']
    nett_margin[i] = margin_df['calc'].sum()
  output2[keys] = {'vogel': vogel, 
                  'table_opt': table_opt, 
                  'nett_margin': nett_margin}

In [ ]:
output2['cost']['nett_margin']

{'AA': 2907800000.0, 'BB': 1708700000.0, 'CC': 2390950000.0}

In [ ]:
# Total nett margin for 'Base of Cost'
sum(output2['cost']['nett_margin'].values())

7007450000.0

In [ ]:
output2['cost']['table_opt']['AA']
# last is row dummy

branch_id,0.0,1.0,2.0,3.0
plant,,,,
0.0,0.0,0.0,3000.0,12000.0
1.0,0.0,15000.0,7000.0,0.0
2.0,20000.0,0.0,0.0,0.0
3.0,0.0,0.0,5000.0,0.0


In [ ]:
output2['cost']['table_opt']['BB']
# last column is dummy

branch_id,0.0,1.0,2.0,3.0,4.0
plant,,,,,
0.0,0.0,0.0,6500.0,7000.0,6500.0
1.0,2000.0,11000.0,2000.0,0.0,0.0
2.0,10000.0,0.0,0.0,0.0,0.0


In [ ]:
output2['cost']['table_opt']['CC']
# last column is dummy

branch_id,0.0,1.0,2.0,3.0,4.0
plant,,,,,
0.0,0.0,0.0,0.0,0.0,12000.0
1.0,0.0,10500.0,0.0,4500.0,0.0
2.0,12000.0,0.0,9000.0,5500.0,500.0


In [ ]:
output2['margin']['nett_margin']

{'AA': 2881800000.0, 'BB': 1670300000.0, 'CC': 2421700000.0}

In [ ]:
# Total nett margin for 'Base on Margin'
sum(output2['margin']['nett_margin'].values())

6973800000.0

In [ ]:
output2['nett']['nett_margin']

{'AA': 2844800000.0, 'BB': 1708500000.0, 'CC': 2454800000.0}

In [ ]:
# Total nett margin for 'Base on nett Margin'
sum(output2['nett']['nett_margin'].values())

7008100000.0

# Using Excel given method

In [ ]:
# this function is base on example calculation in excel that was given
def exc_method(data, supply, demand, supply_id, demand_id, cost, obj='min'):
  nrow = data.groupby(supply_id).count().shape[0]
  ncol = data.groupby(demand_id).count().shape[0]
  data1 = data.copy()
  data1['optimize'] = 0
  for i in range(ncol):
    data2 = data1[data1[demand_id] == i].copy()
    for j in range(nrow):
      # finding minimum/maximum cost value'
      if obj == 'min' :
        minval = min(data2[cost])
      elif obj == 'max':
        minval = max(data2[cost])
      else:
        raise Exception("use objective 'min' [default] for minimalization or 'max' for maximization problem")
      # finding minimum/maximum cost value'
      minval_df = data2[data2[cost] == minval]
      # find optimal allocation
      min_alo = minval_df[[supply,demand]].min(axis=1)
      alo = min_alo.nsmallest(1)
      # assign allocation
      data1.loc[alo.index, 'optimize'] = alo.values[0]

      # updating supply
      row = data1.loc[alo.index, supply_id].values[0]
      row_idx = data1[data1[supply_id] == row].index
      new_supply = data1.loc[alo.index, supply].values[0] - alo.values[0]
      # data1 is for looping all column & row
      data1.loc[row_idx, supply] = new_supply
      # data2 is for looping by row
      data2.loc[alo.index, supply] = new_supply
        
      # updating demand
      col = data2.loc[alo.index, demand_id].values[0]
      col_idx = data2[data2[demand_id] == col].index
      new_demand = data2.loc[alo.index, demand].values[0] - alo.values[0]
      data1.loc[col_idx, demand] = new_demand
      data2.loc[col_idx, demand] = new_demand

      # remove cost in allocation location
      if obj == 'min' :
        limit = np.inf
      else: 
        limit = -np.inf
      data1.loc[alo.index, cost] = limit  
      data2.loc[alo.index, cost] = limit 

  return pd.concat([data, data1['optimize']], axis=1)

In [ ]:
# run excel method to All Data
output3 = {'cost':{}, 'margin':{}, 'nett':{}}

for keys in output3:
  excel = {}
  nett_margin = {}
  table_opt = {}
  type = ['AA', 'BB', 'CC']
  for i in type:
    if keys == 'cost':
      objj = 'min'
    else:
      objj = 'max'
    excel[i] = exc_method(data[i], supply='capacity', demand='demand', 
                   supply_id='plant', demand_id='branch_id', 
                   cost=keys, obj=objj)
    table_opt[i] = pd.pivot_table(data=excel[i], index='plant', 
                                  columns='branch_id', values='optimize')
    margin_df = excel[i].merge(data[i], how='inner')
    # Calculate total net margin
    margin_df['calc'] = margin_df['optimize'] * margin_df['nett']
    nett_margin[i] = margin_df['calc'].sum()
  output3[keys] = {'excel': excel, 
                  'table_opt': table_opt, 
                  'nett_margin': nett_margin}

In [ ]:
output3['cost']['nett_margin']

{'AA': 2904300000, 'BB': 1688900000, 'CC': 2456350000}

In [ ]:
# Total nett margin for min cost base
sum(output3['cost']['nett_margin'].values())

7049550000

In [ ]:
output3['cost']['table_opt']['AA']

branch_id,0,1,2,3
plant,,,,
0,0,0,15000,0
1,0,15000,0,7000
2,20000,0,0,0


In [ ]:
output3['cost']['table_opt']['BB']

branch_id,0,1,2,3
plant,,,,
0,2000,0,8500,7000
1,0,11000,0,0
2,10000,0,0,0


In [ ]:
output3['cost']['table_opt']['CC']

branch_id,0,1,2,3
plant,,,,
0,0,0,9000,3000
1,0,10500,0,4500
2,12000,0,0,2500


In [ ]:
output3['margin']['nett_margin']

{'AA': 2896800000, 'BB': 1688500000, 'CC': 2430550000}

In [ ]:
# Total nett margin for max margin base
sum(output3['margin']['nett_margin'].values())

7015850000

In [ ]:
output3['nett']['nett_margin']

{'AA': 2904300000, 'BB': 1688500000, 'CC': 2430550000}

In [ ]:
# Total nett margin for max nett margin base
sum(output3['nett']['nett_margin'].values())

7023350000

# Optimatization using combination all 3 method for each product and chose the best total net margin

In [ ]:
def find_best_method(cost):
  table = []
  product = ['AA', 'BB', 'CC']
  for i in product:
    LC = output[cost]['nett_margin'][i]
    Vogel = output2[cost]['nett_margin'][i]
    Excel = output3[cost]['nett_margin'][i]
    dict_ = {'LC': int(LC), 'Vogel': int(Vogel), 'Excel': int(Excel)}
    table.append(dict_)
  result = pd.DataFrame(table)
  result.index = product
  result['Best'] = result.max(axis=1)
  result.loc["Total"] = result.sum()
  return result

## Base on Cost

In [ ]:
find_best_method('cost')

,LC,Vogel,Excel,Best
AA,2902800000,2907800000,2904300000,2907800000
BB,1708700000,1708700000,1688900000,1708700000
CC,2381150000,2390950000,2456350000,2456350000
Total,6992650000,7007450000,7049550000,7072850000


- Product AA we choose Vogel method for optimal solution
- Product BB we choose LC cause more simple 
- Product CC we choose Excel method
- with Total Nett Margin = 7.072.850.000



In [ ]:
# Optimal allocation of each product for best method
# last row is dummy 
output2['cost']['table_opt']['AA']

branch_id,0.0,1.0,2.0,3.0
plant,,,,
0.0,0.0,0.0,3000.0,12000.0
1.0,0.0,15000.0,7000.0,0.0
2.0,20000.0,0.0,0.0,0.0
3.0,0.0,0.0,5000.0,0.0


In [ ]:
output['cost']['table_opt']['BB']
#last column is dummy

branch_id,0.0,1.0,2.0,3.0,4.0
plant,,,,,
0.0,0.0,0.0,6500.0,7000.0,6500.0
1.0,2000.0,11000.0,2000.0,0.0,0.0
2.0,10000.0,0.0,0.0,0.0,0.0


In [ ]:
output3['cost']['table_opt']['CC']


branch_id,0,1,2,3
plant,,,,
0,0,0,9000,3000
1,0,10500,0,4500
2,12000,0,0,2500


## Base on Margin (Max)

In [ ]:
find_best_method('margin')

,LC,Vogel,Excel,Best
AA,2876300000,2881800000,2896800000,2896800000
BB,1670300000,1670300000,1688500000,1688500000
CC,2359300000,2421700000,2430550000,2430550000
Total,6905900000,6973800000,7015850000,7015850000


- Product AA we choose Excel method, more simple 
- Product BB we choose Excel method 
- Product CC we choose Vogel method
- with Total Nett Margin = 7.017.600.000



## Base on Nett Margin (Max)

In [ ]:
find_best_method('nett')

,LC,Vogel,Excel,Best
AA,2899300000,2844800000,2904300000,2904300000
BB,1708500000,1708500000,1688500000,1708500000
CC,2341550000,2454800000,2430550000,2454800000
Total,6949350000,7008100000,7023350000,7067600000


- Product AA we choose Excel method 
- Product BB we choose LC method 
- Product CC we choose Vogel method
- with Total Nett Margin = 7.067.600.000



# Conclusion

Finally after some combination we can conclude that the highest Total Nett Margin is produce by "Base on Cost" Optimatization with amount of Total Nett Margin is **7.072.850.000**